## Importando as bibliotecas

In [ ]:
import pandas as pd #para ler, visualizar e printar infos do df
import matplotlib.pyplot as plt #para construir e customizar gráficos
import seaborn as sns #para visualizar uns gráficos
import numpy as np #numpy porque é sempre bom importar numpy né 
import datetime as dt 
import json
from pprint import pprint
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [ ]:
!pip install wordcloud -q

# Ifood Entregador

### Função

In [ ]:
# Essa função pega as variaveis que você precisar do dataFrame, muda seu nome e muda o seu tipo.
def pre_processamento_ifood_entregador(dados, rename_dict, types_dict):
  dados = dados[rename_dict.keys()].copy()
  dados.rename(mapper=rename_dict, axis=1, inplace=True)
  for key,value in types_dict.items():
    dados[key] = dados[key].astype(value)
  return dados

### Importando o arquivo csv

In [ ]:
dados = pd.read_csv('pedidos.csv')
dados.head()

,driver_id,numero_pedido,modo_pagamento,modal,regiao,data_pedido,id_entregador,endereco_origem,dt
0,879587,3037174295,ONLINE,BICYCLE,NOVA IGUACU,2021-03-29 11:19:06.254,a3e47a58-336a-4b66-a330-c6830040ca18,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2021-03-29
1,879587,3037984185,ONLINE,BICYCLE,NOVA IGUACU,2021-03-29 16:34:24.276,a3e47a58-336a-4b66-a330-c6830040ca18,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2021-03-29
2,879587,3037055824,ONLINE,BICYCLE,NOVA IGUACU,2021-03-29 09:11:48.069,a3e47a58-336a-4b66-a330-c6830040ca18,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2021-03-29
3,879587,1935848983,ONLINE,BICYCLE,NOVA IGUACU,2020-06-12 19:20:59.119,a3e47a58-336a-4b66-a330-c6830040ca18,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-06-12
4,879587,2569908938,ONLINE,BICYCLE,NOVA IGUACU,2020-12-01 15:06:43.039,a3e47a58-336a-4b66-a330-c6830040ca18,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-12-01


### Excluir linhas duplicadas

In [ ]:
dados.drop_duplicates(keep='first', inplace=True)

### Variaveis, chamando a função.


In [ ]:

rename = { 
    'modal': 'modal',
    'regiao': 'regiao',		
    'endereco_origem': 'endereco_retirada',
    'data_pedido': 'data_hora'
}

types = {
    'data_hora':'datetime64'
}

dados_pre_processamento = pre_processamento_ifood_entregador(dados, rename, types)
dados = dados_pre_processamento
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332 entries, 0 to 331
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   modal              332 non-null    object        
 1   regiao             332 non-null    object        
 2   endereco_retirada  332 non-null    object        
 3   data_hora          332 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 13.0+ KB


### Organizando os valores pela coluna 'data_hora

In [ ]:
dados.sort_values(['data_hora'], ascending=True, inplace=True, ignore_index=True)
dados.head()

,modal,regiao,endereco_retirada,data_hora
0,BICYCLE,NOVA IGUACU,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-05-16 13:34:38.779
1,BICYCLE,NOVA IGUACU,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-05-17 12:00:32.663
2,BICYCLE,NOVA IGUACU,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-05-17 12:11:20.621
3,BICYCLE,NOVA IGUACU,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-05-17 13:15:51.350
4,BICYCLE,NOVA IGUACU,"Rodovia Presidente Dutra, 13006, Prata, NOVA I...",2020-05-17 14:12:56.318


In [ ]:
dados['data_hora'].min(), dados['data_hora'].max()

(Timestamp('2020-05-16 13:34:38.779000'),
 Timestamp('2021-07-06 17:10:28.091000'))

### Exportando os dados

In [ ]:
dados.to_csv('ifood_entregador.csv', index=False)

# Spotify

### Função

In [ ]:
# Essa função pega as variaveis que você precisar do dataFrame, muda seu nome e muda o seu tipo.
def pre_processamento_spotify(dados, rename_dict, types_dict):
  dados = dados[rename_dict.keys()].copy()
  dados.rename(mapper=rename_dict, axis=1, inplace=True)
  for key,value in types_dict.items():
    dados[key] = dados[key].astype(value)
  return dados

### Impotando arquivo csv

In [ ]:
df = pd.read_json('StreamingHistory0.json')
df.head()

,endTime,artistName,trackName,msPlayed
0,2021-02-07 16:29,Pineapple StormTv,"Poetas no Topo 3.3, Pt. 1",1534
1,2021-02-07 16:29,MV Bill,Estilo Vagabundo,5641
2,2021-02-07 16:29,ADL,Favela Vive 2,14610
3,2021-02-07 16:33,L7NNON,Ninguém (feat. MV BILL),202083
4,2021-02-07 16:36,Pineapple StormTv,"Poetas no Topo 3.3, Pt. 2",179182


### Dicionarios, chamando função.

In [ ]:

rename_spotify = { 
    'artistName':'artista',
    'trackName': 'musica',
    'msPlayed':'tempo',
    'endTime':'data'
}

types_spotify = {
    'data':'datetime64'
}


dados_spotify = pre_processamento_spotify(df, rename_spotify, types_spotify)
df = dados_spotify
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   artista  1802 non-null   object        
 1   musica   1802 non-null   object        
 2   tempo    1802 non-null   int64         
 3   data     1802 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 56.4+ KB


In [ ]:
df['data'].min(), df['data'].max()

(Timestamp('2021-02-07 16:29:00'), Timestamp('2021-07-23 15:04:00'))

In [ ]:
df['tempo']

In [ ]:
df.to_csv('victor_spotify2.csv', index=False)

# Youtube


In [ ]:
#Função que pega uma lista a partir do json. Quando o json precisa de apenas uma chave.
#Funciona para um json que tem uma lista de dicionários
def pega_coluna(json, chave):
        lista = []
        for i in json:
                lista.append(i[chave])
        return lista

In [ ]:
#Função que pega uma lista a partir do json. Quando o json precisa de 2 chaves
#Funciona para um json que tem uma lista de dicionários
#Funciona apenas para o histórico de visualizações. O que não é canal (propaganda) ele retorna como adsense
def pega_coluna_2chave(json, chave1, chave2):
    lista = []
    for i in json:
        if chave1 in i:
            if len(i[chave1]) > 1: lista.pop()
            for k in i[chave1]:
                lista.append(k[chave2])
        else:
            lista.append('adsense')
    return lista

In [ ]:
#Função que pega uma lista a partir do json. Quando o json precisa de 2 chaves
#Serve para arquivo json em que está estruturado com dicionários de dicionários e não lista de dicionários.
def pega_coluna_2chave_dicionarios(json, chave1, chave2):
    lista = []
    for i in json:
        for chave, valor in i[chave1].items():
            if chave == chave2:
                lista.append(valor)
    return lista

In [ ]:
openfile=open("/content/drive/MyDrive/histórico-de-visualização_Celular.json", encoding='UTF-8')
df_youtube_views2 = json.load(openfile)

In [ ]:
openfile=open("/content/drive/MyDrive/histórico-de-visualização_Pc.json", encoding='UTF-8')
df_youtube_views3 = json.load(openfile)

In [ ]:
nome_video_cel = pega_coluna(df_youtube_views2, 'title')
tempo_cel = pega_coluna(df_youtube_views2, 'time')
canal_cel = pega_coluna_2chave(df_youtube_views2, 'subtitles','name')

youtube_celular = pd.DataFrame(data = [canal_cel, nome_video_cel, tempo_cel]).T
youtube_celular.columns=['canal', 'nome_video', 'data']
youtube_celular.head()

,canal,nome_video,data
0,IDEIAS INCRÍVEIS Brincadeiras,Watched 8 IDEIAS IMPRESSIONANTES DE COMO BRINC...,2021-07-18T22:37:01.019Z
1,adsense,Watched PRA VALER E MR.DAN - BIPOLAR ( DVD 15 ...,2021-07-17T01:07:04.129Z
2,Instituto Montanari,Watched 4 PARCELA AUXÍLIO EMERGENCIAL ANTECIPA...,2021-07-13T16:22:41.171Z
3,Instituto Montanari,Watched 4 PARCELA CALENDÁRIO AUXÍLIO EMERGENCI...,2021-07-13T16:22:37.999Z
4,Cortes PodMestre [OFICIAL],Watched O JEITO CERTO DE CHUP4R UMA BCTA l MIA...,2021-07-13T16:19:13.740Z


In [ ]:
nome_video_pc = pega_coluna(df_youtube_views3, 'title')
tempo_pc = pega_coluna(df_youtube_views3, 'time')
canal_pc = pega_coluna_2chave(df_youtube_views3, 'subtitles','name')

youtube_pc = pd.DataFrame(data = [canal_pc, nome_video_pc, tempo_pc]).T
youtube_pc.columns=['canal', 'nome_video', 'data']
youtube_pc.head()

,canal,nome_video,data
0,Descomplicando Bancos,Watched Mercado Crédito: Como Funciona? Vale a...,2021-07-19T20:05:33.358Z
1,Um Eterno Aluno Aprendiz,Watched Sacando o Limite do Mercado Crédito ou...,2021-07-19T19:41:50.214Z
2,Ideris,Watched Como Funciona o Mercado Crédito do Mer...,2021-07-19T19:33:13.188Z
3,Joe Importações e Tutoriais,Watched Mercado Crédito agora você pode sacar ...,2021-07-19T19:28:41.267Z
4,Programação Dinâmica,Watched Introdução a AutoML com PyCaret | Mach...,2021-07-19T16:28:07.089Z


In [ ]:
victor_youtube = pd.concat([youtube_celular, youtube_pc])
victor_youtube.head()

,canal,nome_video,data
0,IDEIAS INCRÍVEIS Brincadeiras,Watched 8 IDEIAS IMPRESSIONANTES DE COMO BRINC...,2021-07-18T22:37:01.019Z
1,adsense,Watched PRA VALER E MR.DAN - BIPOLAR ( DVD 15 ...,2021-07-17T01:07:04.129Z
2,Instituto Montanari,Watched 4 PARCELA AUXÍLIO EMERGENCIAL ANTECIPA...,2021-07-13T16:22:41.171Z
3,Instituto Montanari,Watched 4 PARCELA CALENDÁRIO AUXÍLIO EMERGENCI...,2021-07-13T16:22:37.999Z
4,Cortes PodMestre [OFICIAL],Watched O JEITO CERTO DE CHUP4R UMA BCTA l MIA...,2021-07-13T16:19:13.740Z


In [ ]:
victor_youtube.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2642 entries, 0 to 1685
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   canal       2642 non-null   object
 1   nome_video  2642 non-null   object
 2   data        2642 non-null   object
dtypes: object(3)
memory usage: 82.6+ KB


In [ ]:
victor_youtube['data'] = victor_youtube['data'].astype('datetime64')
victor_youtube.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2642 entries, 0 to 1685
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   canal       2642 non-null   object        
 1   nome_video  2642 non-null   object        
 2   data        2642 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 82.6+ KB


In [ ]:
# str.replace('Watched', '') retira a palavra 'Watched' das linhas da variavel 'nome_vídeo
victor_youtube['nome_video'] = victor_youtube['nome_video'].str.replace('Watched')
victor_youtube.head()

,canal,nome_video,data
0,IDEIAS INCRÍVEIS Brincadeiras,8 IDEIAS IMPRESSIONANTES DE COMO BRINCAR E SE...,2021-07-18 22:37:01.019
1,adsense,PRA VALER E MR.DAN - BIPOLAR ( DVD 15 ANOS ) ...,2021-07-17 01:07:04.129
2,Instituto Montanari,4 PARCELA AUXÍLIO EMERGENCIAL ANTECIPADA? JOÃ...,2021-07-13 16:22:41.171
3,Instituto Montanari,4 PARCELA CALENDÁRIO AUXÍLIO EMERGENCIAL BOLS...,2021-07-13 16:22:37.999
4,Cortes PodMestre [OFICIAL],O JEITO CERTO DE CHUP4R UMA BCTA l MIA LINZ,2021-07-13 16:19:13.740


In [ ]:
victor_youtube.to_csv('youtube_youtube_pessoal.csv', index=False)

# Youtube TV

In [ ]:
openfile=open("/content/drive/MyDrive/histórico-de-visualização_Tv.json", encoding='UTF-8')
df_youtube_views1 = json.load(openfile)

In [ ]:
nome_video = pega_coluna(df_youtube_views1, 'title')
tempo = pega_coluna(df_youtube_views1, 'time')
canal = pega_coluna_2chave(df_youtube_views1, 'subtitles','name')

youtube_tv = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_tv.columns=['canal', 'nome_video', 'data']
youtube_tv.head()

,canal,nome video,data
0,Rafa & Luiz,Watched COMPRAMOS UMA TV DE 86 POLEGADAS! - MU...,2021-07-22T12:51:55.511Z
1,Rafa & Luiz,Watched FIZEMOS UMA ARMADILHA PARA PEGAR O FAC...,2021-07-22T12:34:56.149Z
2,Rafa & Luiz,Watched O MISTÉRIO DA CASA AO LADO! - O FILME!...,2021-07-21T18:22:12.441Z
3,Rafa & Luiz,Watched O FACE TIROU A MÁSCARA! - (TIVEMOS UMA...,2021-07-21T17:56:31.207Z
4,MALOUCOS,Watched A LENDA DA SLIME PERFEITA E SECRETA PA...,2021-07-21T14:01:50.612Z


In [ ]:
# str.replace('Watched', '') retira a palavra 'Watched' das linhas da variavel 'nome_vídeo
youtube_tv['nome_video'] = youtube_tv['nome_video'].str.replace('Watched', '')
youtube_tv.head()

,canal,nome video,data
0,Rafa & Luiz,COMPRAMOS UMA TV DE 86 POLEGADAS! - MUITO GIG...,2021-07-22T12:51:55.511Z
1,Rafa & Luiz,FIZEMOS UMA ARMADILHA PARA PEGAR O FACE! - DE...,2021-07-22T12:34:56.149Z
2,Rafa & Luiz,O MISTÉRIO DA CASA AO LADO! - O FILME! - (COM...,2021-07-21T18:22:12.441Z
3,Rafa & Luiz,O FACE TIROU A MÁSCARA! - (TIVEMOS UMA SURPRESA),2021-07-21T17:56:31.207Z
4,MALOUCOS,A LENDA DA SLIME PERFEITA E SECRETA PARTE 2 -...,2021-07-21T14:01:50.612Z


In [ ]:
youtube_tv.to_csv('tv_youtube_vizualizacao.csv', index = False, header=True)